# 전이 학습
아주 큰 데이터셋을 써서 훈련된 모델의 가중치를 가져와 우리가 해결하려는 과제에 맞게 보정해서 사용하는 것.

전이 학습을 위한 방법
- 특성 추출 (feature extractor)
- 미세 조정 기법 (fine tuning)

## 특성 추출 (feture extractor)

사전 훈련된 모델을 가져온 후 FC 부분만 새로 만든다.
학습 시 FC부분만 학습하고 나머지 계층들은 학습 되지 않도록 한다.

**특성 추출은 목표 특성을 잘 추출했다는 전제하에 좋은 성능을 낼 수 있다. 그게 아니면 미세 조정 기법으로 대체**

### Import lib

In [23]:
# !pip3 install opencv-python 

  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl (55.7 MB)


In [24]:
import os
import time
import copy
import glob
import cv2
import shutil

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

ImportError: dlopen(/Users/goodyoung/Desktop/GIt/myMLstudy/.mlvenv/lib/python3.11/site-packages/cv2/cv2.abi3.so, 2): Symbol not found: __ZNSt3__113basic_filebufIcNS_11char_traitsIcEEE4openEPKcj
  Referenced from: /Users/goodyoung/Desktop/GIt/myMLstudy/.mlvenv/lib/python3.11/site-packages/cv2/.dylibs/libvmaf.1.dylib (which was built for Mac OS X 12.0)
  Expected in: /usr/lib/libc++.1.dylib


### train

In [1]:
data_path = './data/catanddog/train'
    
transform = transforms.Compose(
                [
                    transforms.Resize([256, 256]), #이미지의 크기를 256 by 256으로 조정
                    transforms.RandomResizedCrop(size=(224,224), scale = (0.1,1), ratio=(0.5,2)), # 이미지를 랜덤한 크기 및 비율로 자른다. (데이터 확장 용도.)
                    #면적 비율을 0.1~1 범위 내에서 무작위로 자른다(scale), 면적의 너비와 높이 비율을 0.5~2 범위 내에서 무작위로 조절한다(ratio).
                    transforms.RandomHorizontalFlip(), # 랜덤하게 수평으로 뒤집는다.
                    transforms.ToTensor(), # 이미지 데이터를 텐서로 변환
                ])
train_dataset = torchvision.datasets.ImageFolder( #경로 및 전처리를 함께 포함하여 불러온다.
    data_path,
    transform=transform
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=8,
    shuffle=True
)

print(len(train_dataset))

#### 레이블 정보와 함께 출력

In [ ]:
samples, labels = iter(train_loader).next() #iter()는 전달된 데이터의 반복자를 꺼내 반환하며 next()는 다음에 출력해야 할 요소를 반환한다. train_loader에서 데이터를 하나씩 꺼내옴.
classes = {0:'cat', 1:'dog'}
fig = plt.figure(figsize=(16,24))
for i in range(24):
    a = fig.add_subplot(4,6,i+1)
    a.set_title(classes[labels[i].item()])
    a.axis('off')
    a.imshow(np.transpose(samples[i].numpy(), (1,2,0)))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)

#### modeling

In [27]:
import torchvision.models as models

In [37]:
# resnet18 = models.resnet18() #무작위의 가중치
resnet18 = models.resnet18(pretrained=True) #사전 학습된 모델의 가중치 값

/Users/goodyoung/Desktop/GIt/myMLstudy/.mlvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/goodyoung/Desktop/GIt/myMLstudy/.mlvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [52]:
for i in resnet18.parameters():
    print(f": {i.requires_grad}")

: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: False
: True
: True


In [41]:
# 합성곱층 사용하지만 그 해당 구간에 대해 학습을 하지 않도록 고정시킴.
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
set_parameter_requires_grad(resnet18)

In [47]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [51]:
resnet18.fc = nn.Linear(512, 2) # fc의 out_features가 1000에서 2로 줄어들었다.

In [53]:
# 이 두개의 파라미터를 변환하면 된다.
for name, param in resnet18.named_parameters():
    if param.requires_grad:
        print(name, param.data)

fc.weight tensor([[-0.0380,  0.0094,  0.0075,  ...,  0.0203, -0.0333,  0.0119],
        [ 0.0224, -0.0351,  0.0090,  ..., -0.0219,  0.0241,  0.0183]])
fc.bias tensor([-0.0206, -0.0178])


In [ ]:
# 역전파 중 파라미터들에 대한 변화를 계산할 필요가 없음을 나타낸다.
# transfer learning main code
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(512, 2)
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters()) # 여기서 fc 파라미터들만 준다.
cost = torch.nn.CrossEntropyLoss()
print(model)

In [ ]:
# train
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=13, is_train=True):
    since = time.time()    
    acc_history = []
    loss_history = []
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            model.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1) # 텐서 배열의 최대값이 들어 있는 idx를 반환하는 함수이다.
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)        
        torch.save(model.state_dict(), os.path.join('../chap05/data/catanddog/', '{0:0=2d}.pth'.format(epoch)))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))    
    return acc_history, loss_history

In [56]:
# FC층은 학습을 하도록 학습을 통해 얻어지는 파라미터를 옵티마이저에 전달해서 최종적으로 모델 학습에 사용합니다.
# 학습을 하면 그럼 이 파라미터들이 업데이트 될 것 이다.
params_to_update = []
for name,param in resnet18.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
            
optimizer = optim.Adam(params_to_update)

	 fc.weight
Parameter containing:
tensor([[-0.0380,  0.0094,  0.0075,  ...,  0.0203, -0.0333,  0.0119],
        [ 0.0224, -0.0351,  0.0090,  ..., -0.0219,  0.0241,  0.0183]],
       requires_grad=True)
	 fc.bias
Parameter containing:
tensor([-0.0206, -0.0178], requires_grad=True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
train_acc_hist, train_loss_hist = train_model(resnet18, train_loader, criterion, optimizer, device)

### Test

In [ ]:
test_path = './data/catanddog/train'

transform = transforms.Compose(
                [
                    transforms.Resize(224),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                ])
test_dataset = torchvision.datasets.ImageFolder(
    root=test_path,
    transform=transform
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    num_workers=1,
    shuffle=True
)

print(len(test_dataset))

In [ ]:
def eval_model(model, dataloaders, device):
    since = time.time()    
    acc_history = []
    best_acc = 0.0

    saved_models = glob.glob('../chap05/data/catanddog/' + '*.pth')
    saved_models.sort()
    print('saved_model', saved_models)

    for model_path in saved_models:
        print('Loading model', model_path)

        model.load_state_dict(torch.load(model_path))
        model.eval()
        model.to(device)
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)

            _, preds = torch.max(outputs.data, 1)           
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0
            running_corrects += preds.eq(labels.cpu()).int().sum() # preds 배열과 labels가 일치하는지 검사하는 용도
            #.sum은 예측 결과와 정답이 일치하는 것들의 개수를 합.
            
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print('Acc: {:.4f}'.format(epoch_acc))
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        print()

    time_elapsed = time.time() - since
    print('Validation complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))
    
    return acc_history

In [ ]:
val_acc_hist = eval_model(resnet18, test_loader, device)

In [ ]:
def im_convert(tensor):  
    image=tensor.clone().detach().numpy()   # 메모리에는 새롭게 올리지만 기울기는 전파하지 않겠다.
    image=image.transpose(1,2,0)  
    image=image*(np.array((0.5,0.5,0.5))+np.array((0.5,0.5,0.5)))  
    image=image.clip(0,1)   #0과 1 사이로 제한
    return image  

In [ ]:
# 맞는지 아닌지 출력
classes = {0:'cat', 1:'dog'}

dataiter=iter(test_loader)  
images,labels=dataiter.next()  
output=model(images)  
_,preds=torch.max(output,1) 

fig=plt.figure(figsize=(25,4))  
for idx in np.arange(20):  
    ax=fig.add_subplot(2,10,idx+1,xticks=[],yticks=[])  
    plt.imshow(im_convert(images[idx]))  
    a.set_title(classes[labels[i].item()])
    ax.set_title("{}({})".format(str(classes[preds[idx].item()]),str(classes[labels[idx].item()])),color=("green" if preds[idx]==labels[idx] else "red"))  
plt.show()  
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)

# 미세 조정 기법 (fine-tuning)

특성 추출 기법에서 더 나아가 합성곱층, fc의 가중치를 업데이트하여 훈련시키는 방식입니다.

**즉 사전 학습된 모델을 목적에 맞게 재학습시키거나 학습된 가중치의 일부를 재학습시키는 것이다.**

## **미세 조정 전략**
- 데이터셋 크고, 모델과 유사성이 작은 경우
  - 모델 전체 ***재학습***
- 데이터셋 크고, 모델과 유사성이 큰 경우
  - 합성곱층의 뒷부분 (FC와 가까운 부분)과 데이터 분류기를 학습시킴. 데이터 셋 유사성이 크기 때문이다.
- 데이터셋 작고, 모델과 유사성이 작은 경우
  - 합성곱층의 일부분과 데이터 분류기(FC)를 학습시킨다. 데이터가 적어서 효과가 없을 수 있다. 적당히 설정해야한다.
- 데이터셋 작고, 모델과 유사성이 큰 경우
  - 데이터 분류기만 학습시킨다. 데이터가 적기 때문에 많은 계층에 적용하면 과적합이 발생할 수 있어서 FC에 대해서만 적용한다.